<a href="https://colab.research.google.com/github/Dhanapal-Angamuthu/DeepLearning/blob/main/AIAgent/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install art
!pip install langchain_text_splitters
!pip install langchain_community
!pip install langchain_ollama
!pip install langchain_chroma
!pip install gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.4/610.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.4/379.4 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00


In [ ]:
import argparse
import os
import shutil
import sys
from art import *

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_chroma import Chroma
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM
from langchain.globals import set_verbose, get_verbose
from langchain.prompts import ChatPromptTemplate
from langchain.schema.document import Document
import json
#set_verbose(True)

qachain = None

DB_PATH = "./db"
# Custom JSON loader
class JSONLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        with open(self.file_path, 'r') as file:
            data = json.load(file)
        return data
# Suppress error messages during model loading
def suppress_console_output():
    """Temporarily suppress stdout and stderr."""
    sys.stderr.flush()

    # Save original file descriptors
    stderr_fd = os.dup(2)

    # Open devnull and redirect stdout and stderr
    devnull = os.open(os.devnull, os.O_WRONLY)
    os.dup2(devnull, 2)
    os.close(devnull)

    return stderr_fd  # Return saved descriptors

def restore_console_output(stderr_fd): #stdout_fd, stderr_fd):
    """Restore original stdout and stderr."""
    sys.stdout.flush()
    sys.stderr.flush()
    #os.dup2(stdout_fd, 1)
    os.dup2(stderr_fd, 2)
    #os.close(stdout_fd)
    os.close(stderr_fd)

def llm_learnings():
    global qachain
    ollama = OllamaLLM(base_url='http://localhost:11434', model='llama3')
    #print(ollama("what is AI"))
    urls = [

        "https://en.wikipedia.org/wiki/Artificial_intelligence",
        "https://en.wikipedia.org/wiki/History_of_artificial_intelligence",
        "https://en.wikipedia.org/wiki/Deep_learning"
    ]


    # Initialize an empty list to store documents
    all_documents = []

    loader = WebBaseLoader(urls)
    data = loader.load()

    all_documents.extend(data)

    # Load PDF documents from the data folder
    pdf_loader = PyPDFDirectoryLoader('data')
    pdf_data = pdf_loader.load()
    all_documents.extend(pdf_data)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    all_splits = text_splitter.split_documents(all_documents) #config_data1)
    print("B4 gtp4 embedding")
    # Create a vector store from the document splits
    err_fd = suppress_console_output()
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings()) #HuggingFaceEmbeddings()) #GPT4AllEmbeddings(model_name='mistral-7b-openorca.gguf2.Q4_0.gguf')))
    add_update_chromaDB(all_splits)
    restore_console_output(err_fd)
    print("after gpt4 emb")
    # Set up the QA chain
    qachain = RetrievalQA.from_chain_type(ollama, retriever=vectorstore.as_retriever())
    question = "which switch support 100GbE?"
    return qachain

def add_update_chromaDB(chunks: list[Document]):
    # Load/create database.
    err_fd = suppress_console_output()
    db = Chroma(persist_directory=DB_PATH, embedding_function=GPT4AllEmbeddings())
    restore_console_output(err_fd)

    # Calculate Chunk IDs.
    chunks_and_ids = create_chunkIds(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_and_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print("Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("No new documents to add")


def create_chunkIds(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

def AI_Agent_query(query):
    # Prepare the DB.
    #embedding_function = get_embedding_function()
    #err_fd = suppress_console_output()
    db = Chroma(persist_directory=DB_PATH, embedding_function=GPT4AllEmbeddings())
    #restore_console_output(err_fd)

    # Search the DB.
    results = db.similarity_search_with_score(query, k=3)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query)
    print(prompt)

    model = OllamaLLM(base_url='http://localhost:11434', model='llama3')
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

def get_loader_for_file(file_path):

    file_extension = os.path.splitext(file_path)[1].lower()
    return loaders.get(file_extension, None)

def load_documents_from_directory(directory_path):

    all_documents = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        loader = get_loader_for_file(file_path)

        if loader:
            documents = loader.load(file_path)
            all_documents.extend(documents)

    return all_documents

if __name__ == "__main__":
    tprint("AI Agent",font="fin")
    print("\nLearning is in progress\n")
    qachain = llm_learnings()
    print("Hi There, I am AI Agent. How can I help you today?\n")
    print("Ready for your query to continue or Bye to exit\n")
    while True:
        user_query = input("\n\nEnter Your Query>>> ")
        if user_query=="":
            continue
        if user_query.lower() in ["quit", "bye", "exit"]:
            break
        #resp = qachain.invoke({"query": user_query})
        resp = AI_Agent_query(user_query)
        print("\nAI Agent>>> ", resp) #['result'])



           _____                                 _   
    /\    |_   _|     /\                        | |  
   /  \     | |      /  \     __ _   ___  _ __  | |_ 
  / /\ \    | |     / /\ \   / _` | / _ \| '_ \ | __|
 / ____ \  _| |_   / ____ \ | (_| ||  __/| | | || |_ 
/_/    \_\|_____| /_/    \_\ \__, | \___||_| |_| \__|
                              __/ |                  
                             |___/                   


Learning is in progress

B4 gtp4 embedding
Number of existing documents in DB: 700
No new documents to add
after gpt4 emb
Hi There, I am AI Agent. How can I help you today?

Ready for your query to continue or Bye to exit

